1. 超参数设置

In [ ]:
#其他与GraphSAGE参数设置相同
eps = 0.3

2. 数据集处理与加载

In [ ]:
# 加载数据集
dataset = Planetoid(root='./data', name='Cora')
data = dataset[0]
num_features = dataset.num_features
num_classes = dataset.num_classes

# 获取输入输出维度
input_dim = dataset.num_node_features
output_dim = dataset.num_classes

# 获取训练集和测试集的索引
idx_train = data.train_mask.nonzero(as_tuple=False).view(-1)
idx_test = data.test_mask.nonzero(as_tuple=False).view(-1)


3. 构建卷积层

In [ ]:
class FALayer(MessagePassing):
    def __init__(self, data, num_hidden):
        super(FALayer, self).__init__(aggr='add') # 使用add聚合
        self.data = data
        self.gate = nn.Linear(2 * num_hidden, 1) # 门控机制
        self.row, self.col = data.edge_index # 边的索引
        self.norm_degree = degree(self.row, num_nodes=data.y.shape[0]).clamp(min=1) # 计算度数并进行归一化
        self.norm_degree = torch.pow(self.norm_degree, -0.5)
        nn.init.xavier_normal_(self.gate.weight, gain=1.414) # 初始化权重

    def forward(self, h): 
        h2 = torch.cat([h[self.row], h[self.col]], dim=1) # 拼接节点特征
        g = torch.tanh(self.gate(h2)).squeeze() # 计算门控值
        norm = g * self.norm_degree[self.row] * self.norm_degree[self.col] # 计算归一化值
        return self.propagate(self.data.edge_index, size=(h.size(0), h.size(0)), x=h, norm=norm)

    def message(self, x_j, norm):
        return norm.view(-1,1) * x_j # 消息传递

    def update(self, aggr_out):
        return aggr_out # 更新节点特征


4. 构建模型

In [ ]:
class FAGCN(nn.Module):
    def __init__(self, data, num_features, num_hidden, num_classes, eps):
        super(FAGCN, self).__init__()
        self.eps = eps
        self.layers = nn.ModuleList()
        # 两层FAGCN
        self.layers.append(FALayer(data, num_hidden))
        self.layers.append(FALayer(data, num_hidden))
        # 两层全连接层
        self.t1 = nn.Linear(num_features, num_hidden)
        self.t2 = nn.Linear(num_hidden, num_classes)
        self.reset_parameters()
    def reset_parameters(self):
        # 初始化权重
        nn.init.xavier_normal_(self.t1.weight, gain=1.414)
        nn.init.xavier_normal_(self.t2.weight, gain=1.414)
    def forward(self, h):
        h = torch.relu(self.t1(h))
        raw = h
        # 第一层FAGCN和残差连接
        h = self.layers[0](h)
        h = self.eps * raw + h
        # 第二层FAGCN和残差连接
        h = self.layers[1](h)
        h = self.eps * raw + h
        
        h = self.t2(h)
        return F.log_softmax(h, 1)


5. 模型训练和评估

In [ ]:
for epoch in range(max_epoch):
    # 前向传播
    net.train()
    logp = net(data.x)
    loss = F.nll_loss(logp[idx_train], data.y[idx_train])
    train_acc = accuracy(logp[idx_train], data.y[idx_train])
    # 反向传播
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # 计算准确率
    net.eval()
    logp = net(data.x)
    test_acc = accuracy(logp[idx_test], data.y[idx_test])
    print("Epoch {:03d} | Loss {:.4f} | Train {:.4f} | Test {:.4f} ".format(
        epoch, loss.item(), train_acc, test_acc))
